In [ ]:
using PyPlot

In [7]:
t = 2020:2100 # years
CO₂⁰ = 415    # concentration(ppm) initially

# physical climate parameters
ECS = 3        # Equilibrium Climate Sensitivity
ϵ = ECS/log(2) # Transient Warming Parameter
δT₀⁰ = 1.1;    # Warming: initial, preindustrial

In [8]:
# baseline emissions scenarios
q = fill(2,size(t)) # emissions scenario
c₀ = CO₂⁰ .+ cumsum(q); # cumulative CO2 concentrations (ppm)
δT_0 = δT₀⁰ .+ ϵ.*log.(c₀/CO₂⁰) # warming due to baseline emissions scenario

# economic parameters
β = 2.2 # damage parameter (trillion dolars / C^2)
D_0 = β .* δT₀⁰.^2; # baseline damages
γ = 0 #1. /30. # discount rate
C_ϕ = 400 # trillion $
C_φ = 100 # trillion $
C_χ = 150 # trillion $
C_λ = 150 # trillion $

150

In [9]:
c(ϕ::Array, φ::Array) = (c₀ .- cumsum(q .* φ)) .* (1. .- ϕ);
δT(ϕ::Array, φ::Array) = δT_00 .+ ϵ.*log.(c(ϕ,φ)./c_00);

f_low(α::Array) = (α ./ (1. .+α)).^2 # shape of individual cost functions

D(ϕ::Array, φ::Array, χ::Array, λ::Array) = (1. .- χ) .* β .* δT(ϕ, φ).^2 .* (1. .- λ).^2;
C(ϕ::Array, φ::Array, χ::Array, λ::Array) = C_λ.*f_low(λ) .+ C_ϕ.*f_low(ϕ) .+ C_φ.*f_low(φ) .+ C_χ.*f_low(χ);
T(ϕ::Array, φ::Array, χ::Array, λ::Array) = (
    sum((C(ϕ, φ, χ, λ) .+ D(ϕ, φ, χ, λ)) .* exp.(.-γ.*(t.-t[1]))) .+
    1000. * (sum(diff(ϕ).^2) .+ sum(diff(φ).^2) .+ sum(diff(χ).^2) .+ sum(diff(λ).^2)) .+
    1000. .* (ϕ[1].^2 .+ φ[1].^2 .+ χ[1].^2 .+ λ[1].^2)
)

T (generic function with 1 method)

## Gradient descent cost minimization

In [10]:
function ∇T(ϕ::Array, φ::Array, χ::Array, λ::Array)
    Δα = 1.e-4
    ∂T∂ϕ = zeros(size(ϕ))
    ∂T∂φ = zeros(size(φ))
    ∂T∂χ = zeros(size(χ))
    ∂T∂λ = zeros(size(λ))
    for i=1:length(ϕ)
        δα = zeros(size(ϕ))
        δα[i] = Δα
        ∂T∂ϕ[i] = (T(ϕ.+δα,φ,χ,λ) .- T(ϕ,φ,χ,λ))/Δα
        ∂T∂φ[i] = (T(ϕ,φ.+δα,χ,λ) .- T(ϕ,φ,χ,λ))/Δα
        ∂T∂χ[i] = (T(ϕ,φ,χ.+δα,λ) .- T(ϕ,φ,χ,λ))/Δα
        ∂T∂λ[i] = (T(ϕ,φ,χ,λ.+δα) .- T(ϕ,φ,χ,λ))/Δα
    end
    
    return (∂T∂ϕ, ∂T∂φ, ∂T∂χ, ∂T∂λ)
end

function convergence(∇T, tol = 1.e-6)
    norm = 0.
    for ∂T∂α = ∇T
        norm += sum(∂T∂α.^2)
    end
    return norm < tol
end

convergence (generic function with 2 methods)

#### initial guess

In [11]:
# Example control trajectories
# (picked because cost of controls + cost of controlled damage < uncontrolled damage)
nondim_linear(t::Array) = (t .- t[1])/(t[end] - t[1]);
ϕ = nondim_linear(t)/8.;
φ = nondim_linear(t)/8.;
λ = nondim_linear(t)/8.;
χ = nondim_linear(t)/8.;

MethodError: MethodError: no method matching nondim_linear(::UnitRange{Int64})
Closest candidates are:
  nondim_linear(!Matched::Array) at In[11]:3

In [ ]:
count = 1
while convergence(∇T(ϕ, φ, χ, λ), 1.e-8) == false
    norm = 0.
    for ∂T∂α = ∇T(ϕ, φ, χ, λ)
        norm += sum(∂T∂α.^2)
    end
    print(norm, " ")
    Δs = 1.e-4
    ϕ .-= ∇T(ϕ, φ, χ, λ)[1].*Δs
    φ .-= ∇T(ϕ, φ, χ, λ)[2].*Δs
    χ .-= ∇T(ϕ, φ, χ, λ)[3].*Δs
    λ .-= ∇T(ϕ, φ, χ, λ)[4].*Δs
    count+=1
end

In [ ]:
plot(t, ϕ, label=L"$\phi$ (negative emissions)")
plot(t, φ, label=L"$\varphi$ (emissions reductions)")
plot(t, χ, label=L"$\chi$ (adaptation)")
plot(t, λ, label=L"$\lambda$ (geoengineering)")
legend()

In [ ]:
plot(t,c(ϕ,φ), label=L"$c_{\phi,\varphi}(t)$")
plot(t,, label=L"$c_{0}(t)$")
legend()

In [ ]:
plot(t,δT(ϕ, φ), label=L"\delta T_{\varphi,\phi}")
plot(t,δT_0, label=L"\delta T_{0}")
plot(t,2.0.*ones(size(t)),"k--", label="Paris Goal")
legend()
title("temperature change (since pre-industrial)")

In [ ]:
plot(t, D(ϕ, φ, χ, λ))
title(L"Damages in trillions of $\$$")

In [ ]:
plot(t, C(ϕ, φ, χ, λ) + D(ϕ, φ, χ, λ), label="total cost")
plot(t, C(ϕ, φ, χ, λ), label="controls cost")
plot(t, D(ϕ, φ, χ, λ), label="damage cost")
plot(t, D_0, label="uncontrolled damages")
legend()
title("costs in trillions")